In [157]:
import pandas as pd
import numpy as np
import pickle
import csv

### 3.1

In [158]:
with open('proj2_data.csv', 'r', newline='', encoding='utf-8') as csvfile:
    # read the first 1024 bytes to detect the delimiter
    dialect = csv.Sniffer().sniff(csvfile.read(1024))
csv_delimiter = dialect.delimiter
print('Detected delimiter: ', csv_delimiter)


Detected delimiter:  |


In [159]:
df = pd.read_csv('proj2_data.csv', delimiter=csv_delimiter, decimal='.')
if len(df.columns) <= 1 or 'float64' not in df.dtypes.values:
    df = pd.read_csv('proj2_data.csv', delimiter=csv_delimiter, decimal=",")
    
df.to_pickle('proj2_ex01.pkl')
df

,full_name,field,language,code,task_1,task_2,task_3,tasks_avg,task_grade,jury_score,final_grade
0,Rowan Harrington,drones,python,wej,3.1,2.0,4.4,3.166667,dostateczny,"3,5 pts",dostateczny
1,Nash Wyatt,racing,java,sfe,4.2,2.0,2.0,2.733333,bardzo dobry,5 p,mierny
2,Jadiel Ramirez,media,cplusplus,vaw,4.0,4.9,3.0,3.966667,dobry,3.5,mierny
3,Makaila Atkins,racing,swift,ugt,4.1,5.0,4.8,4.633333,dobry,2,dostateczny
4,Melanie Fuller,racing,python,owb,2.7,2.0,2.0,2.233333,bardzo dobry,pts 2,mierny
5,Layla Woodard,drones,python,wuf,2.8,2.0,2.0,2.266667,bardzo dobry,"3,5p",dostateczny
6,Edwin Hale,media,swift,ceq,4.9,3.0,3.4,3.766667,dostateczny,-1 (dnf),niedostateczny
7,Travis Rocha,robotics,cplusplus,eir,5.0,4.9,5.0,4.966667,dostateczny,0,niedostateczny
8,Ricky Howell,media,java,vos,2.1,2.0,2.0,2.033333,dobry,waiting,niedostateczny
9,Addyson Brennan,drones,java,cov,4.4,3.7,4.0,4.033333,bardzo dobry,-,niedostateczny


### 3.2

In [160]:
scale_dict = {}
file = open('proj2_scale.txt', 'r')

for i in range(1, file.__sizeof__()):
    line = file.readline()
    line = line.strip()
    scale_dict[line] = i
scale_dict.pop('')
print(scale_dict)


{'niedostateczny': 1, 'mierny': 2, 'dostateczny': 3, 'dobry': 4, 'bardzo dobry': 5}


In [161]:
involved_columns = []
for column in df.columns:
    if df[column][0] in scale_dict.keys():
        involved_columns.append(column)
print(involved_columns)
        
df_2 = df.copy()
for key in scale_dict.keys():
    for column in involved_columns:
        df_2[column] = df_2[column].replace(key, scale_dict[key])
df_2.to_pickle('proj2_ex02.pkl')

['task_grade', 'final_grade']


### 3.3

In [162]:
df_3 = df.copy()
for column in involved_columns:
    current_col = df_3[column]
    df_3[column] = pd.Categorical(current_col, categories = scale_dict.keys(), ordered=False)
df_3.to_pickle('proj2_ex03.pkl')

print(df_3.dtypes)


full_name        object
field            object
language         object
code             object
task_1          float64
task_2          float64
task_3          float64
tasks_avg       float64
task_grade     category
jury_score       object
final_grade    category
dtype: object


### 3.4

In [163]:
regex = r'([-]?\d+(?:[.,]\d+)?)'
df_nn_cols = pd.DataFrame()

for column in df.columns:
    current_col = df[column]
    if current_col.dtype not in ['float64', 'int64']:
        if True in current_col.str.contains(regex).values:
            df_nn_cols[column] = pd.to_numeric(current_col.str.replace(',','.').str.extract(regex, expand=False), errors='coerce')

df_nn_cols.to_pickle('proj2_ex04.pkl')
df_nn_cols


/tmp/ipykernel_1668/4200676614.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if True in current_col.str.contains(regex).values:
/tmp/ipykernel_1668/4200676614.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if True in current_col.str.contains(regex).values:


,jury_score
0,3.5
1,5.0
2,3.5
3,2.0
4,2.0
5,3.5
6,-1.0
7,0.0
8,NaN
9,NaN


### 3.5

In [164]:
df_result = pd.DataFrame()

df_list = []
# find columns
for column in df.columns:
    current_col = df[column]
    if current_col.dtype not in ['float64', 'int64'] and len(current_col.unique()) <= 10 and not (False in current_col.str.contains(r'[a-z]').values) and (set(current_col.unique()) - set(scale_dict.keys()) == set(current_col.unique())):
        df_current = pd.get_dummies(current_col)
        df_list.append(df_current)
        
for i, df_i in enumerate(df_list):
    with open(f'proj2_ex05_{i+1}.pkl', 'wb') as f:
        pickle.dump(df_i, f)